# Финальный проект: цыганско-русский словарь #
### Выполнили Кирилл Конча и Елизавета Клыкова (БКЛ181) ###
### Часть 1: парсинг словаря, создание датафрейма ###

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import re
import json
import pandas as pd
from lxml import etree
from collections import OrderedDict
from pymorphy2 import MorphAnalyzer
pm = MorphAnalyzer()

Самоучитель цыганского языка В.В. Шаповала в формате .pdf был распознан, а затем вычитан на предмет опечаток и ошибок распознавания и сохранен в файл 'romadict_txt.txt'.

In [3]:
# считывание файла построчно с удалением пустых строк
lines = []
with open('romadict_txt.txt', 'r', encoding='utf-8') as f:
    for line in f:
        if line != '\n':
            lines.append(line.strip('\n'))

# деление на две части (кривое, но самое простое решение)
rom_lines = lines[0:2730]
rus_lines = lines[2731:]

Словарные статьи русской и цыганской статей имеют немного разную структуру (например, расположение тегов и т.д.). Функция parse_line принимает на вход одну строку словаря и язык, к которому относится слово, и с помощью регулярных выражений выделяет в ней несколько частей: слово, его перевод, словарные пометы (пока все вместе, деление позже), этимологию и специальную часть, указывающую, является ли слово формой какого-то другого слова. Любая из последних трех частей может отсутствовать.

Функция возвращает список вида \[слово, язык, перевод, теги, лемма с комментариями, этимология\].

In [4]:
def parse_line(line, lang):
    word_info = []
    line = re.sub(';', ',', line)
    if lang == 'rom':
        # находим слово с тегами, разделяем
        word_with_tags = re.search('(.+?) — ', line).group(1)
        if ' (' in word_with_tags and '.' in word_with_tags:
            word = re.search(r'(.+?) [(]', word_with_tags).group(1)
            tags = re.search(r' [(](.+?)[)]', word_with_tags).group(1)
        else:
            word = word_with_tags
            tags = ''
        word = re.sub(' и ', ', ', word)

        # получаем перевод и этимологию
        definition = re.split(' — ', line)[-1]
        if ' | ' in definition:
            etymology = re.search(r'\| (.+)', definition).group(1)
            transl = definition.replace(etymology, '').strip(r' | ')
        else:
            etymology = ''
            transl = definition

        # получаем лемму, если есть, а затем отделяем перевод от
        # указания на лемму
        form_of = ''
        if ' к ' in transl:
            if '.' in transl and ',' not in transl:
                form_of = re.search('([^,]*?к .+)', transl).group(1)
            elif '.' in transl or ', ' in transl:
                form_of = re.search(', ([^,]*?к .+)', transl).group(1)
            transl = re.sub(re.escape(form_of), '', transl)
            transl = re.sub(',$', '', transl.strip())

        # часть тегов не удается получить на предыдущем этапе,
        # т.к. они записаны у перевода, а не исходного слова
        if transl.endswith(')'):
            add_tags = re.search(r'([^\(]+)?\)$', transl).group(1)
            bad_tags = ['букв.', 'т.е.', ')', 'ср.', 'Дж.', 'разг.', 'ж.',
                        'мн.', 'м.', 'ед.']
            if ('.' in add_tags or 'счетное слово' in add_tags) and \
              not any(t in add_tags for t in bad_tags):
                if tags:
                    tags = tags + ', ' + add_tags
                else:
                    tags = add_tags
            transl = re.sub(re.escape(add_tags), '', transl)
            transl = re.sub(',$', '', (re.sub(r' \(\)', '', transl)).strip())

        if '.' in transl:
            last = transl.split(',')[-1].strip()
            more_tags = ['общая ф.', 'завис. ф.', 'прош.']
            if last == 'зват. ф.' or any(t in last for t in more_tags):
                if tags:
                    tags = tags + ', ' + last
                else:
                    tags = last
                transl = re.sub(re.escape(last), '', transl)
                transl = re.sub(',$', '', transl.strip())

    if lang == 'ru':
        # в русской части теги обычно не у слова, а у перевода
        word = re.search('(.+?) — ', line).group(1)
        transl_with_tags = re.search(' — (.+)', line).group(1)
        if ' (' in transl_with_tags and '.' in transl_with_tags:
            transl = re.search(r'(.+?) [(]', transl_with_tags).group(1)
            tags = re.search(r' [(](.+?)[)]', transl_with_tags).group(1)
        else:
            transl = transl_with_tags
            tags = ''
        transl = re.sub(' и ', ', ', transl)
        # этимология у русских слов не указана
        etymology = ''

        # иногда теги все же бывают у слова
        form_of = ''
        if ' к ' in word:
            if '.' in word and ',' not in word:
                form_of = re.search('([^,]*?к .+)', word).group(1)
            elif '.' in word:
                form_of = re.search(', ([^,]*?к .+)', word).group(1)
            word = re.sub(re.escape(form_of), '', word)
            word = re.sub(',$', '', word.strip())

        # отлавливаем пропущенные теги
        if word.endswith(')') and '.' in word:
            add_tags = re.search(r'\(.+\)$', word).group()
            word = re.sub(re.escape(add_tags), '', word).strip()
            add_tags = re.sub(r'[\(\)]', '', add_tags)
            if tags:
                tags = tags + ', ' + add_tags
            else:
                tags = add_tags

    # добавляем всю полученную информацию в список
    word_info.extend((word, lang, transl, tags, form_of, etymology))

    return word_info

42:15: E127 continuation line over-indented for visual indent


Функция parse_all_lines парсит все строки на указанном языке, используя функцию parse_line. Полученная информация сохраняется в список x_words, где x - аббревиатура языка (rom или ru).

In [5]:
def parse_all_lines(lines, lang):
    words = []
    for line in lines:
        if len(re.findall(' — ', line)) == 1:
            word_info = parse_line(line, lang)
            words.append(word_info)
        elif len(re.findall(' — ', line)) > 1:
            parts = re.split('; ', line)
            for part in parts:
                word_info = parse_line(part, lang)
                words.append(word_info)
    return words

In [6]:
# получаем информацию о словарных статьях
rom_words = parse_all_lines(rom_lines, 'rom')
rus_words = parse_all_lines(rus_lines, 'ru')

# сохраняем в общий список
all_words = rom_words + rus_words

In [7]:
# создаем датафрейм на основе полученного списка
columns = ['word',  'lang', 'translation', 'tags', 'form_of', 'etymology']
df = pd.DataFrame(all_words, columns=columns)

Следующий этап - разделение тегов на категории (часть речи, число, род и т.д.).

In [8]:
# нужные пометы сохраняем в списки
pos_tags = ['сз.', 'част.', 'мест.', 'межд.', 'нар.', 'гл.', 'предл.',
            'прич.', 'прил.', 'числ.', 'деепр.', 'сравн.']
verb_tags = ['повел.', 'неп.', 'пер.', 'пер./неп.', 'прош.']
case_tags = ['тв.', 'дат.', 'вин.', 'зват. ф.']
gender_tags = ['м.', 'ж.', 'м./ж.']
number_tags = ['ед.', 'мн.']

In [9]:
# записываем теги (в т.ч. пустые) из столбца 'tags' в список
word_tags = df['tags'].tolist()

Функция split_tags принимает список значений колонки 'tags' и разделяет теги на 7 частей (не обязательно все они присутствуют в каждой статье, может не быть вообще ни одной): часть речи, глагольные теги (переходность и наклонение, но только повелительное), падеж, род, число и прочие пометы, объединенные в категорию other. Функция возвращает список словарей.

In [10]:
def split_tags(word_tags):
    full_tag_info = []

    for word in word_tags:
        # создаем словарь с информацией о тегах слова
        word_info = OrderedDict()
        tag_types = ['pos', 'vform', 'case', 'gender',
                     'number', 'other']
        for ttype in tag_types:
            word_info[ttype] = []

        # здесь можно добавить проверку на (не)пустоту поля с тегами,
        # но данных не очень много и скорость не страдает

        # делим строку с тегами по запятым
        word = re.sub(' и ', ', ', word)
        tags = [t.strip() for t in word.split(',')]

        # проверяем теги по одному
        # функцией не получается, много специфичных моментов
        for tag in tags:
            if tag in pos_tags:
                word_info['pos'].append(tag)
            elif tag in case_tags:
                word_info['case'].append(tag)
            elif tag in verb_tags:
                word_info['vform'].append(tag)
                if tag == 'повел.' or tag == 'прош.':
                    word_info['pos'].append('гл.')
            elif tag in gender_tags:
                word_info['gender'].append(tag)
            elif tag in number_tags:
                word_info['number'].append(tag)
            elif 'гл.' in tag:
                word_info['pos'].append('гл.')
                vtags = re.sub('гл. ', '', tag).split('/')
                for v in vtags:
                    if v.strip() in verb_tags:
                        word_info['vform'].append(v.strip())
            elif 'предл.' in tag:
                word_info['pos'].append('предл.')
                word_info['other'].append(re.sub('предл. ', '', tag))
            elif tag.startswith('ж.') or tag.startswith('м.'):
                gender = tag.split()[0]
                word_info['gender'].append(gender)
            elif tag.endswith('ед.') or tag.endswith('мн.'):
                num = tag.split()
                if 'только' in tag:
                    number = 'только ' + num[-1]
                else:
                    number = num[-1]
                word_info['number'].append(number)
            else:
                word_info['other'].append(tag)

        # перед записью в общий список объединяем
        # значения словаря word_info через ', '
        for key in word_info:
            word_info[key] = ', '.join(word_info[key])
        full_tag_info.append(word_info)

    return full_tag_info

In [11]:
full_tag_info = split_tags(word_tags)

# добавляем полученную информацию в датафрейм
# исходная колонка со всеми тегами сохраняется
tag_df = pd.DataFrame(full_tag_info)
df = pd.concat([df, tag_df], axis=1)

### Экспериментальная часть ###
Мы попробовали добавить частеречные теги к словам, для которых они не указаны в исходном словаре. В качестве парсера был выбран pymorphy: он дает хорошую точность и работает быстрее, чем mystem (если говорить о модулях для юпитера), при этом парсить его выдачу гораздо проще, чем выдачу консольного mystem'a (который работает намного быстрее на больших текстовых файлах).

Для цыганских слов выбирался тег русского перевода. Если вариантов перевода было несколько, учитывался первый, если pymorphy предалагал несколько разборов, выбирался наиболее вероятный. В случае, когда перевод слова состоял из нескольких слов (не считая пометы и примечания), устанавливалось pos = 'unknown'.

**Важно:** никакие другие теги, кроме частеречных, мы не добавляли, так как даже в случае с частями речи возникало довольно много неточностей и/или случаев, когда приходилось ставить тег 'unknown'. Дальнейшая автоматическая разметка, на мой (Лиза К.) взгляд, нецелесообразна: надежнее доразметить вручную, тем более, что в получившемся датафрейме всего 4045 строк.

Мы считали оригинальную разметку приоритетной (сохраняли ее там, где она была, и только приводили к универсальному виду - об этом см. ниже).

In [12]:
words_with_pos = df[['word', 'lang', 'translation', 'pos']].values.tolist()

In [13]:
parts_of_speech = []
prons = ['я', 'ты', 'он', 'она']

for word in words_with_pos:
    # если часть речи уже есть в списке, ничего не парсим
    if word[-1] != '':
        pos = word[-1]
    else:
        pos = None
        # дальше сложности, потому что русский перевод не всегда
        # начинается с нужного слова, да и вообще не всегда есть
        if word[1] == 'rom':
            w = word[2]
        elif word[1] == 'ru':
            w = word[0]
        if w.startswith('см.'):
            pos = 'unknown'
        else:
            t = re.sub(r'\(.+?\)', '', re.sub(r'\[.+?\]', '', w)).strip()
            if ',' in t:
                t = t.split(',')[0].strip()
            if ' ' not in t:
                t = re.sub('!', '', t)
                pos = pm.parse(t)[0].tag.POS
            else:
                parts = [p for p in t.split() if p not in prons]
                # если после всех проверок перевод все еще содержит
                # больше одного слова, то часть речи не определяется
                # такие случаи нужно размечать вручную
                if len(parts) == 1:
                    pos = pm.parse(parts[0])[0].tag.POS
                else:
                    pos = 'unknown'
    # сюда попадают случаи, с которыми не справился pymorphy
    # их мало (2-3 случая)
    if not pos:
        pos = 'unknown'
    parts_of_speech.append(pos)

In [14]:
# записываем в датафрейм обновленные части речи
df['pos'] = parts_of_speech

### Универсализация грамматических помет ###
За основу был взят тегсет, принятый в Pymorphy (https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html), c определенными изменениями: число помет было уменьшено за счет объединения некоторых категорий (ADJF и ADJS - ADJ, PRTF и PRTS - PRTCP). Теги падежей, рода, числа, переходности/непереходности взяты из тегсета НКРЯ (https://ruscorpora.ru/new/corpora-morph.html) с небольшими изменениями - например, нет второго родительного и т.д.

Здесь важно сказать, что в источнике очень ограниченный набор помет, относящихся к падежу существительных и времени глагола: они указаны лишь для небольшого числа слов.

In [15]:
# правила унификации, для каждой категории свое
universal_pos = {'сз.': 'CONJ', 'част.': 'PRCL', 'гл.': 'VERB',
                 'мест.': 'PRON', 'межд.': 'INTJ', 'нар.': 'ADVB',
                 'предл.': 'PREP', 'прич.': 'PRTCP', 'прил.': 'ADJ',
                 'числ.': 'NUMR', 'деепр.': 'GRND', 'сущ.': 'NOUN',
                 'сравн.': 'COMP', 'ADJS': 'ADJ', 'COMP': 'COMP',
                 'ADJF': 'ADJ', 'ADVB': 'ADVB', 'CONJ': 'CONJ',
                 'GRND': 'GRND', 'INFN': 'VERB', 'NOUN': 'NOUN',
                 'NPRO': 'PRON', 'NUMR': 'NUMR', 'PRCL': 'PRCL',
                 'PRED': 'ADVB', 'PREP': 'PREP', 'PRTF': 'PRTCP',
                 'PRTS': 'PRTCP', 'VERB': 'VERB', 'INTJ': 'INTJ',
                 'unknown': 'unknown'}
universal_cases = {'им.': 'NOM', 'род.': 'GEN', 'дат.': 'DAT',
                   'вин.': 'ACC', 'тв.': 'INS', 'местн.': 'LOC',
                   'зват. ф.': 'VOC'}
universal_vtags = {'пер.': 'TRAN', 'неп.': 'INTR',
                   'повел.': 'IMPER', 'прош.': 'PAST'}
universal_gender = {'м.': 'm', 'ж.': 'f', 'м./ж.': 'm.;f.'}
universal_number = {'ед.': 'sg', 'мн.': 'pl', 'только мн.': 'pl'}

Функция unify_tags принимает на вход список тегов из датафрейма и правило, согласно которому эти теги должны быть преобразованы, и возвращает видоизмененный список тегов.

In [16]:
def unify_tags(tags, universal):
    new_tags = []
    for tag in tags:
        value = []
        if tag:
            for v in [t.strip() for t in tag.split(',')]:
                value.append(universal[v])
            new_tag = ';'.join(value)
        else:
            new_tag = ''
        new_tags.append(new_tag)
    return new_tags

In [17]:
# наверное, можно упростить
new_pos = unify_tags(df['pos'].values.tolist(), universal_pos)
new_vtags = unify_tags(df['vform'].values.tolist(), universal_vtags)
new_cases = unify_tags(df['case'].values.tolist(), universal_cases)
new_gender = unify_tags(df['gender'].values.tolist(), universal_gender)
new_number = unify_tags(df['number'].values.tolist(), universal_number)

In [18]:
# кажется, можно красивее, но я умираю
df['pos'] = new_pos
df['vform'] = new_vtags
df['case'] = new_cases
df['gender'] = new_gender
df['number'] = new_number

In [19]:
df

,word,lang,translation,tags,form_of,etymology,pos,vform,case,gender,number,other
0,а,rom,а,"сз., част.",,,CONJ;PRCL,,,,,
1,ав,rom,"приди, будь",повел.,,снскр. ājā- приходить,VERB,IMPER,,,,
2,"аваса, редко васа",rom,будем,,,,VERB,,,,,
3,"авир, вавир",rom,"другой, другая",мест.,,,PRON,,,,,
4,аври,rom,вон,межд.,,,INTJ,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
4040,язык,ru,чиб,ж.,,,NOUN,,,f,,
4041,языкастый,ru,чибало,прил.,,,ADJ,,,,,
4042,язычок,ru,чибори,ж.,,,NOUN,,,f,,
4043,яичница,ru,дзэвэлы,ж.,,,NOUN,,,f,,


### Дополнения после защит ###
Борис Леонидович посоветовал выделить из статей типа "аваса, редко васа" помету "редко" и убрать ее из поля word, что и было сделано с помощью кода ниже.

In [20]:
for j in range(0, len(df)):
    if 'редк.' in df['word'][j] or 'редко' in df['word'][j]:
        switch = 0
        string = ''
        other = ''
        for i in df['word'][j].split(' '):
            if i != 'редк.' and i != 'редко' and switch == 0:
                new = i.strip(',')
                if len(string) == 0:
                    string = string + new
                else:
                    string = string + ', ' + new
            if i == 'редк.' or i == 'редко':
                switch = 1
                other = other + 'редко'
            if switch == 1 and i != 'редк.' and i != 'редко':
                other = other + ' ' + i
        df['word'][j] = string
        if len(df['other'][j]) == 0:
            df['other'][j] = other
        else:
            df['other'][j] = df['other'][j] + '; ' + other

Сохраняем готовый датафрейм в формате .tsv.

In [21]:
df.to_csv('romadict_dataframe.tsv', sep='\t', index=False)

### Часть 2: создание json-файла для веб-версии ###
Веб-версия словаря доступна по ссылке https://romadict.linghub.ru.

Возьмем из датафрейма все колонки и сделаем из них список словарей.

In [22]:
list_d = []
for i in range(0, len(df)):
    dic = {}
    dic['id'] = i
    dic['word'] = df['word'][i]
    dic['lang'] = df['lang'][i]
    dic['translation'] = df['translation'][i]
    dic['_tags'] = df['tags'][i]
    dic['etymology'] = df['etymology'][i]
    dic['pos'] = df['pos'][i]
    dic['_form_of'] = df['form_of'][i]
    dic['_vform'] = df['vform'][i]
    dic['_case'] = df['case'][i]
    dic['_gender'] = df['gender'][i]
    dic['_number'] = df['number'][i]
    dic['_other'] = df['other'][i]
    list_d.append(dic)

Приведем данные к нужному для фреймворка виду:

In [23]:
data = {'data': list_d}

Сохраним в json-файл:

In [24]:
with open('data.json', 'w', encoding='utf-8') as fp:
    json.dump(data, fp, ensure_ascii=False)

### Часть 3: создание xml-файла в TEI ###
Считываем датафрейм (считывание дает возможность не перезапускать весь код, который был выше):

In [25]:
df_file = pd.read_csv('romadict_dataframe.tsv', sep='\t')
df_file.fillna('', inplace=True)

In [26]:
# делим на цыганскую и русскую части
rom_rows = df_file[df_file['lang'] == 'rom'].values.tolist()
rus_rows = df_file[df_file['lang'] == 'ru'].values.tolist()

Для создания XML-файла используется модуль lxml. Сначала оформляем шапку (doctype добавим в самом конце, это проще сделать вручную).

In [27]:
root = etree.Element('TEI', xmlns='http://www.tei-c.org/ns/1.0')
root.set('{http://www.w3.org/XML/1998/namespace}lang', 'ru')
header = etree.SubElement(root, 'teiHeader')

Первая часть - fileDesc; она содержит заголовок, информацию об авторах, данные словаря (кол-во статей, место и дату публикации) и источник, откуда взяты данные.

In [28]:
file_desc = etree.SubElement(header, 'fileDesc')

title_stmt = etree.SubElement(file_desc, 'titleStmt')
title = etree.SubElement(title_stmt, 'title')
title.text = 'Цыганско-русский и русско-цыганский словарь'

resp_stmt = etree.SubElement(title_stmt, 'respStmt')
resp1 = etree.SubElement(resp_stmt, 'resp')
resp1.text = 'Авторы:'
name1 = etree.SubElement(resp_stmt, 'name')
name1.text = 'Елизавета Клыкова '
email1 = etree.SubElement(name1, 'email',
                          value='eaklykova@edu.hse.ru')
email1.text = 'eaklykova@edu.hse.ru,'

resp2 = etree.SubElement(resp_stmt, 'resp')
name2 = etree.SubElement(resp_stmt, 'name')
name2.text = 'Кирилл Конча '
email2 = etree.SubElement(name2, 'email',
                          value='majortomblog@gmail.com')
email2.text = 'majortomblog@gmail.com'

length = etree.SubElement(file_desc, 'extent')
length.text = '4045 статей'

pub_stmt = etree.SubElement(file_desc, 'publicationStmt')
pub_place = etree.SubElement(pub_stmt, 'pubPlace')
ref = etree.SubElement(
    pub_place, 'ref', target='https://github.com/eaklykova/romadict')
ref.text = 'https://github.com/eaklykova/romadict'
date = etree.SubElement(pub_stmt, 'date', when='2021')
date.text = 'март 2021 г.'

source_desc = etree.SubElement(file_desc, 'sourceDesc')
source = etree.SubElement(source_desc, 'bibl')
source.text = '''Шаповал, В.В. Самоучитель цыганского языка \
(русска рома: севернорусский диалект). Учебное пособие. \
М.: АСТ, 2007.'''

Следующая часть - encodingDesc, где хранится описание проекта и ссылки на курс и руководителей.

In [29]:
encoding_desc = etree.SubElement(header, 'encodingDesc')
project_desc = etree.SubElement(encoding_desc, 'projectDesc')
par1 = etree.SubElement(project_desc, 'p')
par1.text = '''Проект выполнен в рамках дисциплины «Теоретическая \
и прикладная лексикография», которая преподается в Школе лингвистики \
НИУ ВШЭ (Факультет гуманитарных наук). '''
course_link = etree.SubElement(
    par1, 'ref',
    target='https://www.hse.ru/ba/ling/courses/375293130.html')
course_link.text = '''Cсылка на курс: \
https://www.hse.ru/ba/ling/courses/375293130.html.'''

par2 = etree.SubElement(project_desc, 'p')
par2.text = 'Преподаватели дисциплины:'
prof_list = etree.SubElement(par2, 'list', rend='bulleted')

prof1 = etree.SubElement(prof_list, 'item')
prof1_link = etree.SubElement(
    prof1, 'ref', target='https://www.hse.ru/staff/olesar')
prof1_link.text = 'О.Н. Ляшевская (руководитель проекта)'

prof2 = etree.SubElement(prof_list, 'item')
prof2_link = etree.SubElement(
    prof2, 'ref', target='https://www.hse.ru/org/persons/34792977')
prof2_link.text = 'В.Ю. Апресян'

prof3 = etree.SubElement(prof_list, 'item')
prof3_link = etree.SubElement(
    prof3, 'ref', target='https://www.hse.ru/org/persons/34616715')
prof3_link.text = 'Б.Л. Иомдин'

# не нашла ссылку :(
prof4 = etree.SubElement(prof_list, 'item')
prof4_link = etree.SubElement(prof4, 'ref')
prof4_link.text = 'Е.В. Еникеева'

par3 = etree.SubElement(project_desc, 'p')
par3.text = '''В рамках проекта отрывок из Самоучителя \
цыганского языка был оцифрован и преобразован в электронный \
словарь. Онлайн-версия словаря размещена по ссылке '''
dict_link = etree.SubElement(
    par3, 'ref', target='https://romadict.linghub.ru')
dict_link.text = 'https://romadict.linghub.ru.'

par4 = etree.SubElement(project_desc, 'p')
par4.text = 'Репозиторий проекта на Github: '
github_link = etree.SubElement(
    par4, 'ref', target='https://github.com/eaklykova/romadict')
github_link.text = 'https://github.com/eaklykova/romadict.'

Наконец, содержательная часть со словарными статьями:

In [30]:
text = etree.SubElement(root, 'text')
text.set('{http://www.w3.org/XML/1998/namespace}lang', 'ru')
body = etree.SubElement(text, 'body')

Функция make_tei принимает список строк датафрейма, относящихся к одному языку, и создает элемент \<entry\> с вложенными тегами. Большая часть тегов не добавляется, если в датафрейме нет значения, которое должно в них располагаться.

In [31]:
def make_tei(rows):
    for row in rows:
        # в кач-ве id берем только первое слово без запятых и пробелов
        # добавляем язык, чтобы минимизировать вероятность совпадений
        word_parts = row[0].split(',')[0].strip()
        word_id = word_parts.split()[0] + '_' + row[1]

        # одно entry - одно слово/лексема
        entry = etree.SubElement(body, 'entry', type='mainEntry')
        entry.set('{http://www.w3.org/XML/1998/namespace}lang', row[1])
        entry.set('{http://www.w3.org/XML/1998/namespace}id', word_id)

        # само слово
        form = etree.SubElement(entry, 'form', type='word')
        orth = etree.SubElement(form, 'orth')
        orth.text = row[0]

        # gramGrp содержит все грамматические пометы
        gram_grp = etree.SubElement(entry, 'gramGrp')

        # часть речи
        pos = etree.SubElement(gram_grp, 'pos', type='pos', value=row[6])
        pos.text = row[6]

        # глагольные теги
        if row[7]:
            vt = etree.SubElement(
                gram_grp, 'v_info', type='v_info', value=row[7])
            vt.text = row[7]

        # падеж
        if row[8]:
            gender = etree.SubElement(
                gram_grp, 'case', type='case', value=row[8])
            gender.text = row[8]

        # род
        if row[9]:
            gender = etree.SubElement(
                gram_grp, 'gen', type='gen', value=row[9])
            gender.text = row[9]

        # число
        if row[10]:
            number = etree.SubElement(
                gram_grp, 'num', type='num', value=row[10])
            number.text = row[10]

        # другие пометы (столбец 'other')
        if row[11]:
            other = etree.SubElement(entry, 'notes')
            other.text = row[11]

        # если слово является формой лексемы и это указано в словаре,
        # добавляем специальное поле и ссылку на лексему
        if row[4]:
            hyper_entry = etree.SubElement(entry, 'xr', type='cf')
            # составляем id: с помощью регулярных выражений
            # находим нужную часть строки
            if ')' not in row[4]:
                hyper_id = row[4].split()[-1]
            else:
                hyp = re.search(r'(.+)\(', row[4]).group(1)
                hyper_id = hyp.strip().split()[-1]

            # добавляем язык, как и в случае с word_id
            # поле form_of всегда ведет к цыганской статье
            link_to_hyper = '#' + hyper_id + '_rom'
            form_of = etree.SubElement(hyper_entry, 'ref',
                                       target=link_to_hyper)
            form_of.text = row[4]

        # хочется, чтобы перевод позволял перейти на аналогичную статью
        # другого языка, поэтому нужна ссылка; это сложно, потому что
        # перевод плохо парсится и нет гарантии, что это слово есть
        # среди статей другого языка -> нужна ручная доработка
        senses = etree.SubElement(entry, 'sense')
        cit = etree.SubElement(senses, 'cit', type='trans')
        translation = etree.SubElement(cit, 'quote', value=row[2])
        if row[2]:
            # удаляем ненужные элементы
            tr_link = re.sub(r'\[.+?\]', '', row[2])
            tr_link = re.sub('см. ', '', tr_link.split(',')[0].strip())
            tr_parts = tr_link.split()
            # выбираем нужную часть и подставляем в id
            if len(tr_parts) == 2 and \
              pm.parse(tr_parts[0])[0].tag.POS == 'PREP':
                key = tr_parts[1]
            else:
                key = tr_parts[0]
            link_to_transl = '#' + key
            # добавляем аббревиатуру языка, для рус. слов 'rom' и наоборот
            if row[1] == 'rom':
                link_to_transl += '_ru'
            else:
                link_to_transl += '_rom'
            transl = etree.SubElement(
                translation, 'ref', target=link_to_transl)
            transl.text = row[2]
        else:
            # в очень редких случаях перевода нет (только теги)
            translation.text = '—'

        # этимология
        if row[5]:
            etymology = etree.SubElement(entry, 'etym', type='etym')
            etymology.text = row[5]

87:15: E127 continuation line over-indented for visual indent


Заполняем файл по частям, добавляя между частями соответствующие заголовки:

In [32]:
rom_title = etree.SubElement(body, 'div', type='title')
rom_text = etree.SubElement(rom_title, 'p')
rom_text.text = 'ЦЫГАНСКО-РУССКИЙ СЛОВАРЬ'
make_tei(rom_rows)

In [33]:
rus_title = etree.SubElement(body, 'div', type='title')
rus_text = etree.SubElement(rus_title, 'p')
rus_text.text = 'РУССКО-ЦЫГАНСКИЙ СЛОВАРЬ'
make_tei(rus_rows)

Записываем готовый xml-документ в файл "romadict_xml.xml":

In [34]:
tree = root.getroottree()
tree.write('romadict_xml.xml', encoding='utf-8', pretty_print=True)

Добавим в готовый xml-файл переводы строки между леммами для удобства восприятия (способ неизящный, но через lxml это делать сложно - если нужно изменить один конкретный отступ, то нужно прописывать и все остальные).

In [35]:
with open('romadict_xml.xml', 'r+', encoding='utf-8') as f:
    xml_file = f.read()
    xml_with_blanks = re.sub('<entry', '\n\n    <entry', xml_file)
    doctype = '''<?xml version='1.0' encoding='UTF-8'?>
<?xml-stylesheet type='text/css' href='romadict.css'?>
<!DOCTYPE TEI SYSTEM 'freedict-P5.dtd'>\n\n'''
    xml_with_doctype = doctype + xml_with_blanks
    f.seek(0)
    f.write(xml_with_doctype)
    f.truncate()